In [1]:
%runfile ../General-Tools/LazyList.sage
%runfile ../TS-Methods/MonoidTS.sage
%runfile ../TS-Methods/RingTS.sage
%runfile ../Sequence-Methods/Berlekamp.sage
%runfile ../TS-Methods/ComplexTS.sage

In [2]:
TS = ComplexTannakianSymbols()
ts = TS.parseSymbol

In [3]:
import pickle
import json

In [4]:
import random

maxEvenDim = 10
maxOddDim = 10
elementRange = (1, 5)

precision = 20

length = 50000

data = [None] * length
symbols = [None] * length

def printProgress(i, max):
    sys.stdout.write("\r%d (%d%%)" % (i, 100 * i / max))
    sys.stdout.flush()

In [7]:

random.seed(42)
for i in range(length):
    evendim = random.randint(0, maxEvenDim)
    odddim  = random.randint(0, maxOddDim)
    sym = []
    for j in range(evendim):
        sym.append([random.randint(*elementRange), 1])
    for j in range(odddim):
        sym.append([random.randint(*elementRange), -1])
    symbol = TS.createElement(sym)
    data[i] = (symbol.getBellCoefficients().toList(precision), symbol.evendimension())
    symbols[i] = symbol
    
    if i != 0 and (i % 10) == 0:
        printProgress(i, length)

printProgress(length, length)

50000 (100%)

In [8]:
with open('recursionData.json', 'w') as f:
    json.dump(map(lambda (l, r): (map(float, l), int(r)), data), f)

In [5]:
with open('recursionData.json') as f:
    data2 = map(lambda (l, r): (map(float, l), int(r)), json.load(f))

In [6]:
training_size = 45000

In [7]:
testing_size = 5000

In [8]:
properData = map(lambda (coeffs, evendim): (coeffs, [0]*(evendim) + [1] + [0]*(maxEvenDim - evendim)), data2)

In [9]:
training_data = properData[0:training_size]

In [10]:
testing_data = properData[training_size:training_size + testing_size]

In [11]:
import tensorflow as tf
import numpy as np

In [12]:
x = tf.placeholder(tf.float32, [None, precision])

In [13]:
W = tf.Variable(tf.zeros([precision, precision], tf.float32))
W1 = tf.Variable(tf.zeros([precision, precision], tf.float32))
W2 = tf.Variable(tf.zeros([precision, maxEvenDim + 1], tf.float32))
b = tf.Variable(tf.zeros([maxEvenDim + 1]))

In [14]:
y1 = tf.nn.softmax(tf.matmul(x, W))
y2 = tf.nn.softmax(tf.matmul(y1, W1))
y = tf.nn.softmax(tf.matmul(y2, W2) + b)

In [15]:
y_ = tf.placeholder(tf.float32, [None, maxEvenDim + 1])

In [16]:
# cross entropy might not be ideal
cost = -tf.reduce_mean(tf.reduce_sum(y * y_))

In [17]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

In [18]:
init = tf.global_variables_initializer()

In [19]:
sess = tf.Session()

In [20]:
sess.run(init)

In [21]:
printProgress(0, training_size)
for i, (bellCoeffs, evendim) in enumerate(training_data):
    sess.run(train_step, feed_dict={x: [bellCoeffs], y_: [evendim]})
    
    #if sess.run(tf.reduce_sum(tf.cast(tf.is_nan(W), tf.float32))) > 0:
        #print 
        #print "became nan at data[", i, "]"
        #break
        
    if (i + 1) % 50 == 0:
        printProgress(i + 1, training_size)

0 (0%)

50 (0%)

100 (0%)

150 (0%)

200 (0%)

...

44800 (99%)

44850 (99%)

44900 (99%)

44950 (99%)

45000 (100%)

In [22]:
testing_data_input = map(lambda (bellCoeffs, evendim): bellCoeffs, testing_data)
testing_data_output = map(lambda (bellCoeffs, evendim): evendim, testing_data)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print sess.run(accuracy, feed_dict={x: testing_data_input, y_: testing_data_output})

0.2584


In [23]:
sess.close()

In [24]:
sess.run(y, feed_dict={x: [map(lambda n: n, range(20))]})

RuntimeError: Attempted to use a closed Session.

In [25]:
training_data[16][0]

[1.0,
 1.0,
 9.0,
 45.0,
 225.0,
 1125.0,
 5625.0,
 28125.0,
 140625.0,
 703125.0,
 3515625.0,
 17578125.0,
 87890625.0,
 439453125.0,
 2197265625.0,
 10986328125.0,
 54931640625.0,
 274658203125.0,
 1373291015625.0,
 6866455078125.0]

In [39]:
sess.run(W)